In [ ]:
pip install requests-toolbelt

In [1]:
import io
import cv2
import requests
import numpy as np
from PIL import Image
from requests_toolbelt.multipart.encoder import MultipartEncoder

In [2]:
img_set = cv2.imread("card sets.jpg")
img_gray = cv2.imread("card sets.jpg", 0)

card_type_img = []
# img_set = cv2.resize(img_set, (900, 400))
# src_hsv = cv2.cvtColor(img_set, cv2.COLOR_BGR2HSV)

# h, s, v = cv2.split(src_hsv)

_, dst = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
# contours, hier = cv2.findContours(dst, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# print(len(contours))
# print(hier)


# cv2.drawContours(img_set, contours, -1, (0, 0, 255), 1, cv2.LINE_AA, hier)



cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

# print(cnt)
# print(labels.shape)
# print(stats)
# print(centroids)


img_gray = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

src = img_set[23:23+310, 23:23+222]
for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]

    # 노이즈 제거
    if area < 10000:
        continue
    if area > 100000:
        continue
    
#     cv2.rectangle(img_set, (x, y, w, h), (0, 0, 255))
    dst = img_set[y:y+310, x:x+221].copy()
    dst = cv2.resize(dst, (148, 206), cv2.INTER_AREA)
    card_type_img.append(dst)
    
#     cv2.imshow('img3', src)
#     cv2.waitKey()

card_type_img.pop()
card_type_img.pop()
# cv2.imshow('v', v)
# cv2.imshow('img1', img_set)
# cv2.imshow('img2', dst)

# for idx in card_type_img:
# #     print(idx.shape)
#     cv2.imshow('img', idx)
#     cv2.waitKey()
# cv2.waitKey()
# cv2.destroyAllWindows()

print(len(card_type_img))

52


In [3]:
type_card = ['AC', '2C', '3C', '4C', '5C', '6C', '7C', '8C', '9C', '10C', 'JC', 'QC', 'KC',
                'AH', '2H', '3H', '4H', '5H', '6H', '7H', '8H', '9H', '10H', 'JH', 'QH', 'KH',
                'AS', '2S', '3S', '4S', '5S', '6S', '7S', '8S', '9S', '10S', 'JS', 'QS', 'KS',
                'AD', '2D', '3D', '4D', '5D', '6D', '7D', '8D', '9D', '10D', 'JD', 'QD', 'KD']

dict_card = dict(zip(type_card, card_type_img))

In [4]:
# cv2.imshow('img', dict_card['KC'])
# cv2.waitKey()
# cv2.destroyAllWindows()

In [5]:
# bg = np.zeros((206+50, (148+3)* 7, 3), np.uint8)
# w = 10
# h = 120
# bg = np.ones((300, 300), np.uint8)*255
# str = 'HELLO'
# cv2.putText(bg, str , (w, h), 
#              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
# cv2.imshow('bg', bg)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [25]:
NUM_SORT = {'A' : 1, '2' : 2, '3' : 3, '4' : 4, '5' : 5, '6' : 6,
            '7' : 7, '8' : 8, '9' : 9, '10' : 10, 'J' : 11, 'Q' : 12, 'K' : 13}
NUM_RANK = {'A' : 13, '2' : 1, '3' : 2, '4' : 3, '5' : 4, '6' : 5,
            '7' : 6, '8' : 7, '9' : 8, '10' : 9, 'J' : 10, 'Q' : 11, 'K' : 12}
HAND_RANK = {'Royal Flush' : 9, 'Straight Flush' : 8, 'Four of a kind' : 7, 
            'Full House' : 6, 'Flush' : 5, 'Straight' : 4, 'Three of a kind' : 3, 
            'Two pair' : 2, 'One pair' : 1}
STRAIGHT_COMB = ['A2345', '23456', '34567', '45678', '56789',
                 '678910', '78910J', '8910JQ', '910JQK']
ROYAL_FLUSH = '10JQKA'

In [21]:
# 내 패를 확인해서 숫자와 패턴이 얼마나 있는지 딕셔너리로 반환
def check_hands(num_list_f, pattern_list_f):
    num_count_dict = {}
    pattern_count_dict = {}
    for i in num_list_f:
        try:
            num_count_dict[i] += 1
        except:
            num_count_dict[i] = 1
    for j in pattern_list_f:
        try:
            pattern_count_dict[j] += 1
        except:
            pattern_count_dict[j] = 1
    return num_count_dict, pattern_count_dict

# 내 패를 확인해 가장 좋은 패 조합을 출력
def print_hands(ncount_f, pcount_f):
    # 입력으로 받아온 딕셔너리(숫자와 패턴의 수)를 인덱스 값으로 오름차순 정렬
    dict_n = dict(sorted(ncount_f.items(), key=lambda x: x [1]))
    dict_p = dict(sorted(pcount_f.items(), key=lambda x: x [1]))
    
    
    pair = False              # 페어의 여부를 확인
    straight = False         # 스트레이트의 여부를 확인
    flush = False            # 플러쉬의 여부를 확인
    hand_ranking = []     # 내 패의 조합들을 넣을 리스트
    high_card = []          # 내 패의 조합들의 가장 높은 숫자카드를 넣을 리스트
    temp_list = []           # 임시로 저장에 쓸 리스트
#     comb = None
    key_list = num_list.copy()
    
    # 숫자 조합 확인(스트레이트 제외)
    for key, idx in dict_n.items():
        # 같은 숫자가 4개 있으면
        if idx == 4:
            # 포카드
            hand_ranking.append('Four of a kind')
            high_card.append(key)
        # 같은 숫자가 3개있고
        elif idx == 3:
            # 페어 상태라면
            # (딕셔너리를 인덱스 기준으로 오름차순 정렬했기 때문에
            # 풀하우스이면서 페어상태가 아닐수는 없음)
            if pair:
                # 풀하우스
                hand_ranking.append('Full House')
                high_card.append(key)
            # 페어 상태가 아니라면
            else:
                # 트리플
                hand_ranking.append('Three of a kind')
                high_card.append(key)
        # 같은 숫자가 2개있고
        elif idx == 2:
            # 페어 상태라면
            if pair:
                # 투페어
                hand_ranking.append('Two pair')
                high_card.append(key)
            # 페어 상태가 아니면
            else:
                # 원페어
                hand_ranking.append('One pair')
                high_card.append(key)
                # 페어상태를 True로
                pair = True
    
    # 스트레이트 검출을 위해 숫자들을 모두 받은 리스트를 오름차순으로 정렬
    n = len(key_list)
    for i in range(len(key_list)-1):
        n -= 1
        for j in range(n):
            if NUM_SORT[key_list[j+1]] < NUM_SORT[key_list[j]]:
                temp_key = key_list[j]
                key_list[j] = key_list[j+1]
                key_list[j+1] = temp_key
    for i in key_list:
        try:
            i in comb
        except:
            comb = i
        else:
            if not(i in comb):
                comb += i
    # 카드 문양 확인, 플러쉬 검출
    for key, idx in dict_p.items():
        # 같은 문양이 5개 이상 있다면
        if idx >= 5:
            # 플러쉬 상태를 True로
            flush = True
            # 그리고 플러쉬 상태인 카드들의 숫자를 추가
            for key_n, idx_p in card_dict.items():
                if key in idx_p:
                    temp_list.append(key_n)
        # 마찬가지로 카드들을 오름차순으로 정렬
        n = len(temp_list)
        for i in range(len(temp_list)-1):
            n -= 1
            for j in range(n):
                if NUM_SORT[temp_list[j+1]] < NUM_SORT[temp_list[j]]:
                    temp_idx = temp_list[j]
                    temp_list[j] = temp_list[j+1]
                    temp_list[j+1] = temp_idx
        for i in temp_list:
            try:
                i in find_flush_num
            except:
                find_flush_num = i
            else:
                if not(i in find_flush_num):
                    find_flush_num += i
    # 만약 플러쉬 상태의 숫자들중 A가 있다면 로얄 플러쉬 검출을 위해
    # royal_num에 숫자들의 조합에서 A를 가장 뒤로 빼준 값을 넣음
    try:
        'A' in find_flush_num
    except:
        pass
    else:
        if 'A' in find_flush_num:
            royal_list = list(find_flush_num)
            royal_list.append(royal_list.pop(0))
            for i in royal_list:
                try:
                    royal_num += i
                except:
                    royal_num = i
    # 스트레이트 검출
    for elem in STRAIGHT_COMB:
        # 내 패가 플러쉬 상태 이고
        if flush:
            try:
                ROYAL_FLUSH in royal_num
            except:
                pass
            else:
                # royal_num에 로얄 스트레이트 조합이 들어가 있다면
                if ROYAL_FLUSH in royal_num:
                    # 로얄 플러쉬이다
                    hand_ranking.append('Royal Flush')
                    high_card.append('A')
                    break
            finally:
                if elem in find_flush_num:
                    hand_ranking.append('Straight Flush')
                    high_card.append(list(elem)[-1])
                else:
                    hand_ranking.append('Flush')
                    if 'A' in find_flush_num:
                        high_card.append('A')
                    else:
                        high_card.append(list(find_flush_num)[-1])
        # 플러쉬 상태가 아니고
        else:
            # 조합에 연속된 숫자가 있다면
            if elem in comb:
                # 스트레이트
                hand_ranking.append('Straight')
                high_card.append(list(elem)[-1])
    # str을 분리시켜 넣었기 때문에 '10'이 '1', '0'으로 분리가 되어
    # '0'이 들어왔을 경우 '0' 대신 '10'을 넣음
    if '0' in high_card:
        high_card[high_card.index('0')] = '10'
    
    if len(hand_ranking) > 1:
        max_rank = hand_ranking[0]
        for i in hand_ranking:
            if HAND_RANK[max_rank] < HAND_RANK[i]:
                max_rank = i
        max_num = high_card[hand_ranking.index(max_rank)]
    elif len(hand_ranking) == 1:
        max_rank = hand_ranking[0]
        max_num = high_card[0]
        if len(high_card) != 1:
            for i in high_card:
                if NUM_RANK[max_num] < NUM_RANK[i]:
                    max_num = i
    elif len(hand_ranking) == 0:
        max_rank = 'High Card'
        max_num = num_list[0]
        for i in num_list:
            if NUM_RANK[max_num] < NUM_RANK[i]:
                max_num = i
                
    print(high_card)
#     if max_num == '0':
#         max_num = '10'
        
    result = max_num + ' ' + max_rank
    return result

In [22]:
# card_type = ['AC'] # 하이카드

# card_type = ['9C', '9D', '9H', '6C', '2H'] # 트리플
# card_type = ['JH', 'JC', '7H', '7D', '7C'] # 풀하우스
# card_type = ['4D', '4C', 'JC', 'JH', '9H'] # 투 페어
# card_type = ['6C', '6H', '3C', 'QD', '10D'] # 원 페어
# card_type = ['10D', '10S', '10H', '10C', '4D'] # 포카드
# card_type = ['AC', 'KD', '4C', '10D', '8H'] # 하이카드

# card_type = ['3H', '4C', '5D', '6C', '7S', '9D'] # 스트레이트
# card_type = ['AH', '2C', '3D', '4C', '5S', '9D'] # 스트레이트
# card_type = ['6H', '7C', '8D', '9C', '10S', '9D'] # 스트레이트
card_type = ['8H', '7C', '6D', '9C', '10S', '9D'] # 스트레이트

# card_type = ['2H', '6H', '9H', 'QH', 'KH', 'AH'] # 플러쉬
# card_type = ['AS', '3S', '4S', '5S', '6S', '7H'] # 플러쉬

# card_type = ['3S', '4S', '5S', '6S', '7S', '8H', '3H'] # 스트레이트 플러쉬
# card_type = ['AS', '5S', '6S', '7S', '8S', '9S'] # 스트레이트 플러쉬
# card_type = ['AD', '5S', '6S', '7S', '9S', '8S'] # 스트레이트 플러쉬

# card_type = ['10H', 'JH', 'QH', 'KH', 'AH', '5C', '2H'] # 로얄 플러쉬


# print(card_type)

In [23]:
num_list = []
pattern_list = []
card_dict = {}
for idx in card_type:
    str_list = list(idx)
#     print('str_list = ', str_list)
    if len(str_list) == 3:
        str_list[0] += list(idx)[1]
        str_list[1] = str_list[2]
        str_list.pop()
    num_list.append(str_list[0])
    pattern_list.append(str_list[1])

# print('num_list = ', num_list)
# print('pattern_list = ', pattern_list)

for i in range(len(num_list)):
#     print(i)
    try:
        card_dict[num_list[i]]
    except:
        card_dict[num_list[i]] = pattern_list[i]
    else:
        temp = card_dict[num_list[i]]
        card_dict[num_list[i]] = []
        card_dict[num_list[i]].append(temp)
        card_dict[num_list[i]].append(pattern_list[i])

ncount, pcount = check_hands(num_list, pattern_list)

res = print_hands(ncount, pcount)

print(res)

['9', '10']
10 Straight


In [12]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('failed')
    sys.exit()
    
card_type = []
confidence = {}

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fps = int(cap.get(cv2.CAP_PROP_FPS))
capture = None
check = 0

# fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# edge = False
# flip = False


while True:
    ret, frame = cap.read()
    
    if not ret:
        print('failed')
        break
        
    img = frame
    

    
    if check == 0 and not(capture is None):
        card_type = []
        image = cv2.cvtColor(capture, cv2.COLOR_BGR2RGB)
        pilImage = Image.fromarray(image)

        # Convert to JPEG Buffer
        buffered = io.BytesIO()
        pilImage.save(buffered, quality=100, format="JPEG")

        # Build multipart form and post request
        m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})

        response = requests.post("https://detect.roboflow.com/play_cards_standard/1?api_key=hn2gJ5RSbbaI5Az5o7MT", data=m, headers={'Content-Type': m.content_type})
        result = response.json()['predictions']
        # print(type(result))
        #  print(result)
        # display(result)
        check += 1
        for idx in result:
            x = int(idx['x'] - (idx['width']/2))
            y = int(idx['y'] - (idx['height']/2))
            text = "{:0.2f}%, {}".format(idx['confidence']*100, idx['class'])
            cv2.rectangle(capture, (x, y, idx['width'], idx['height']), (0, 0, 255), 1)
            cv2.putText(capture, text , (x, y-5) ,cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            confidence[idx['class']] = idx['confidence']*100

            if not (idx['class'] in card_type):
                card_type.append(idx['class'])
        
    
    cv2.imshow('img', img) 
    
    if card_type:
        num_list = []
        pattern_list = []
        card_dict = {}
        for idx in card_type:
            str_list = list(idx)
        #     print('str_list = ', str_list)
            if len(str_list) == 3:
                str_list[0] += list(idx)[1]
                str_list[1] = str_list[2]
                str_list.pop()
            num_list.append(str_list[0])
            pattern_list.append(str_list[1])

        # print('num_list = ', num_list)
        # print('pattern_list = ', pattern_list)

        for i in range(len(num_list)):
        #     print(i)
            try:
                card_dict[num_list[i]]
            except:
                card_dict[num_list[i]] = pattern_list[i]
            else:
                temp = card_dict[num_list[i]]
                card_dict[num_list[i]] = []
                card_dict[num_list[i]].append(temp)
                card_dict[num_list[i]].append(pattern_list[i])

        ncount, pcount = check_hands(num_list, pattern_list)

        result_text = print_hands(ncount, pcount)
        
    if check == 1 and not(capture is None):
        cv2.imshow('detect', capture)
        check += 1
        start_w = 0
        if card_type:
            bg = np.zeros((206+50, ((148+3)* len(card_type)-3), 3), np.uint8)
            for idx in card_type:
                h, w = dict_card[idx].shape[:2]
                txt = '{}({:0.2f}%)'.format(idx, confidence[idx])
                bg[0:h, start_w:start_w + w] = dict_card[idx]
                cv2.putText(bg, txt , (start_w + int(w/5), h+15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(bg, result_text , (10 , 246),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                start_w = start_w + w + 3
            cv2.imshow('bg', bg)
        else:
            try:
                cv2.destroyWindow('bg')
            except:
                print('no window')
        
    key = cv2.waitKey(50)
    
    if key == 27:
        break
    elif key == ord('d'):
        capture = img
#         detection = not detection
        check = 0

cap.release()
cv2.destroyAllWindows()